In [3]:
import bs4 as bs
import datetime as dt
import pandas as pd
import os
import pandas_datareader.data as web
import pickle
import requests
from dateutil.relativedelta import relativedelta, FR

In [6]:
end_date = pd.Timestamp(pd.to_datetime('today').strftime("%m/%d/%Y"))
start_date = end_date - relativedelta(years=3)

In [7]:
def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text[:-1]
        tickers.append(ticker)
    with open("sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)
    return tickers

# save_sp500_tickers()
def get_data_from_yahoo(start_date, end_date, reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')

    start = start_date
    end = end_date
    for ticker in tickers:
        # just in case your connection breaks, we'd like to save our progress!
        ticker = ticker.replace('.', '-')
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            try:
                df = web.DataReader(ticker, 'yahoo', start, end)
                df.reset_index(inplace=True)
                df.set_index("Date", inplace=True)
                df.to_csv('stock_dfs/{}.csv'.format(ticker))
                print('Create {}'.format(ticker))
            except:
                print('Drop {}'.format(ticker))
                pass
        else:
            print('Already have {}'.format(ticker))

get_data_from_yahoo(start_date, end_date, reload_sp500=True)

Already have MMM
Already have ABT
Already have ABBV
Already have ABMD
Already have ACN
Already have ATVI
Already have ADBE
Already have AMD
Already have AAP
Already have AES
Already have AMG
Already have AFL
Already have A
Already have APD
Already have AKAM
Already have ALK
Already have ALB
Already have ARE
Already have ALXN
Already have ALGN
Already have ALLE
Already have AGN
Already have ADS
Already have LNT
Already have ALL
Already have GOOGL
Already have GOOG
Already have MO
Already have AMZN
Already have AMCR
Already have AEE
Already have AAL
Already have AEP
Already have AXP
Already have AIG
Already have AMT
Already have AWK
Already have AMP
Already have ABC
Already have AME
Already have AMGN
Already have APH
Already have ADI
Already have ANSS
Already have ANTM
Already have AON
Already have AOS
Already have APA
Already have AIV
Already have AAPL
Already have AMAT
Already have APTV
Already have ADM
Already have ARNC
Already have ANET
Already have AJG
Already have AIZ
Already have 

In [10]:
def compile_data():
    with open("sp500tickers.pickle", "rb") as f:
        tickers = pickle.load(f)

    main_df = pd.DataFrame()

    for count, ticker in enumerate(tickers):
        try:
            df = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
            df.set_index('Date', inplace=True)
            df = pd.DataFrame(df['Adj Close'])
            df.rename(columns={'Adj Close': ticker}, inplace=True)

            if main_df.empty:
                main_df = df
            else:
                main_df = main_df.join(df, how='outer')
        except:
            pass

        if count % 10 == 0:
            print(count)

    print(main_df.head())
    main_df.to_csv('sp500_joined_closes.csv')

compile_data()

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
                   MMM        ABT       ABBV        ABMD         ACN  \
Date                                                                   
2016-10-03  161.533096  40.076836  55.869083  127.800003  114.227455   
2016-10-04  158.432373  39.652992  55.665863  127.370003  111.751228   
2016-10-05  159.087601  40.255795  56.010460  127.449997  111.722878   
2016-10-06  158.395462  40.472420  55.515656  128.500000  112.015869   
2016-10-07  158.109406  40.472420  55.604008  128.470001  109.634163   

                 ATVI        ADBE   AMD         AAP        AES    ...      \
Date                                                              ...       
2016-10-03  43.259327  108.449997  6.95  146.162888  10.964231    ...       
2016-10-04  43.288761  108.389999  6.97  145.764984  10.507021    ...  